In [4]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomAffine, ColorJitter

# --- Этап 1. Загрузка и предобработка данных ---


train_dir = 'dataset/ogyeiv2/train'
val_dir = 'dataset/ogyeiv2/test'

# Трансформации для обучающего набора данных
train_transforms = Compose([
    RandomAffine(degrees=45, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Трансформации для валидационного набора данных (без аугментации)
val_transforms = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Создаем датасеты, применяя трансформации
train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
val_dataset = ImageFolder(root=val_dir, transform=val_transforms)

# Создание DataLoader'ов
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


class_names = train_dataset.classes
print(f"Количество классов: {len(class_names)}")
print(f"Количество изображений в обучающем датасете: {len(train_dataset)}")
print(f"Количество изображений в валидационном датасете: {len(val_dataset)}")

Количество классов: 84
Количество изображений в обучающем датасете: 2352
Количество изображений в валидационном датасете: 504


In [6]:
import torch.nn as nn
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights

# --- Этап 2. Объявление модели ---

# Загружаем предварительно обученную модель
model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1)

# Замораживаем все веса в модели
for param in model.parameters():
    param.requires_grad = False

# Получаем количество классов из загрузчика данных
num_classes = len(train_dataset.classes)

# Заменяем последний слой (классификатор) на новый
# in_features для mobilenet_v3_small равно 576
model.classifier = nn.Sequential(
    nn.Linear(in_features=576, out_features=1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(in_features=1024, out_features=num_classes)
)


total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Общее количество параметров: {total_params}")
print(f"Количество обучаемых параметров: {trainable_params}")


Общее количество параметров: 1603956
Количество обучаемых параметров: 676948


In [7]:
import torch.optim as optim
import time

# --- Этап 3. Обучение или дообучение ---


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Обучение будет происходить на устройстве: {device}")
model.to(device)

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Гиперпараметры обучения
EPOCHS = 15
best_vloss = float('inf')
model_save_path = 'meds_classifier.pt'

# Цикл обучения
for epoch in range(EPOCHS):
    start_time = time.time()
    
    # Фаза обучения
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # Фаза валидации
    model.eval()
    running_vloss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_vloss += loss.item() * inputs.size(0)

    epoch_vloss = running_vloss / len(val_loader.dataset)
    
    # Вывод результатов эпохи
    end_time = time.time()
    print(f"Эпоха {epoch+1}/{EPOCHS} | "
          f"Train Loss: {epoch_loss:.4f} | "
          f"Val Loss: {epoch_vloss:.4f} | "
          f"Время: {end_time - start_time:.2f} сек")

    # Сохранение лучшей модели
    if epoch_vloss < best_vloss:
        best_vloss = epoch_vloss
        torch.save(model.state_dict(), model_save_path)
        print(f"  Модель сохранена в '{model_save_path}' (лучшая Val Loss: {best_vloss:.4f})")

print("\nОбучение завершено.")
print(f"Файл с лучшей моделью сохранен как '{model_save_path}'")

Обучение будет происходить на устройстве: cpu


KeyboardInterrupt: 